In [4]:
import numpy as np

# ============================
# 1. Diccionario básico
# ============================
# Diccionario de traducción palabra por palabra
vocab_eng = ['i', 'you', 'he', 'she', 'we', 'they', 'eat', 'drink', 'see', 'like',
             'am', 'are', 'is', 'the', 'a', 'an', 'apple', 'water', 'book', 'house']

vocab_esp = ['yo', 'tú', 'él', 'ella', 'nosotros', 'ellos', 'como', 'bebo', 'veo', 'me gusta',
             'soy', 'eres', 'es', 'el', 'un', 'una', 'manzana', 'agua', 'libro', 'casa']

# Crear diccionarios para traducción
eng2esp = dict(zip(vocab_eng, vocab_esp))

# Crear diccionarios de índices para embedding
eng2idx = {word: idx for idx, word in enumerate(vocab_eng)}
esp2idx = {word: idx for idx, word in enumerate(vocab_esp)}
idx2eng = {idx: word for idx, word in enumerate(vocab_eng)}
idx2esp = {idx: word for idx, word in enumerate(vocab_esp)}

# Traductor palabra por palabra
def translate(sentence):
    words = sentence.lower().split()
    translated = []
    for word in words:
        if word in eng2esp:
            translated.append(eng2esp[word])
        else:
            translated.append(f"[{word}]")  # palabra desconocida
    return ' '.join(translated)

# ============================
# 2. Utilidades del modelo
# ============================

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return angle_rads

def scaled_dot_product_attention(Q, K, V, mask=None):
    matmul_qk = np.matmul(Q, K.transpose(0, 2, 1))
    dk = K.shape[-1]
    scaled_attention_logits = matmul_qk / np.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = softmax(scaled_attention_logits)
    output = np.matmul(attention_weights, V)
    return output, attention_weights

def multi_head_attention(Q, K, V, num_heads):
    batch_size, seq_len, d_model = Q.shape
    depth = d_model // num_heads
    outputs = []

    for _ in range(num_heads):
        q = Q.copy()
        k = K.copy()
        v = V.copy()
        head, _ = scaled_dot_product_attention(q, k, v)
        outputs.append(head)

    concat = np.concatenate(outputs, axis=-1)
    return concat


# ============================
# 3. Encoder y Decoder
# ============================

def encoder(X, num_heads=2, d_model=24):
    seq_len = X.shape[0]
    X_with_pos = X + positional_encoding(seq_len, d_model)
    attention = multi_head_attention(X_with_pos[np.newaxis], X_with_pos[np.newaxis], X_with_pos[np.newaxis], num_heads)
    return attention.squeeze()

def decoder(Y, encoder_output, num_heads=2, d_model=24):
    seq_len = Y.shape[0]
    Y_with_pos = Y + positional_encoding(seq_len, d_model)
    masked_attention = multi_head_attention(Y_with_pos[np.newaxis], Y_with_pos[np.newaxis], Y_with_pos[np.newaxis], num_heads)
    enc_dec_attention = multi_head_attention(masked_attention, encoder_output[np.newaxis], encoder_output[np.newaxis], num_heads)
    return enc_dec_attention.squeeze()


# ============================
# 4. Funciones de entrada y salida
# ============================

def embed_sentence(sentence, vocab_dict, d_model):
    tokens = sentence.lower().split()
    embedded = np.zeros((len(tokens), d_model))
    for i, word in enumerate(tokens):
        idx = vocab_dict.get(word, 0)
        one_hot = np.zeros(len(vocab_dict))
        one_hot[idx] = 1
        embedded[i] = np.pad(one_hot, (0, d_model - len(vocab_dict)), 'constant')
    return embedded

def decode_output(output, vocab_dict):
    result = []
    for vec in output:
        idx = np.argmax(vec[:len(vocab_dict)])
        result.append(idx2esp.get(idx, '?'))
    return ' '.join(result)


# ============================
# 5. Interacción con el usuario
# ============================

def run_basic_translator():
    print("Traductor básico de inglés a español (usa solo palabras conocidas)")
    print("Palabras permitidas:", ', '.join(vocab_eng))

    while True:
        inp = input("\nEscribe una frase en inglés (o 'salir'): ").strip().lower()
        if inp == "salir":
            break
        output = translate(inp)
        print("Entrada:", inp)
        print("Traducción aproximada:", output)

def run_advanced_translator():
    d_model = 24
    print("Traductor avanzado de inglés a español (solo usa palabras como: i, eat, apple, etc.)")
    print("Palabras permitidas:", ', '.join(vocab_eng))

    while True:
        input_sentence = input("\nEscribe una frase en inglés (o 'salir'): ").strip().lower()
        if input_sentence == 'salir':
            break

        words = input_sentence.split()
        if all(word in vocab_eng for word in words):
            X = embed_sentence(input_sentence, eng2idx, d_model)
            encoded = encoder(X)
            decoded = decoder(X, encoded)
            translated = decode_output(decoded, esp2idx)

            print("Entrada:", input_sentence)
            print("Traducción aproximada:", translated)
        else:
            print("❌ Solo se permiten palabras del vocabulario definido.")

if __name__ == "__main__":
    print("Selecciona el tipo de traductor:")
    print("1: Traductor básico (palabra por palabra)")
    print("2: Traductor avanzado (con modelo de atención)")

    choice = input("Ingresa tu opción (1 o 2): ")

    if choice == "1":
        run_basic_translator()
    elif choice == "2":
        run_advanced_translator()
    else:
        print("Opción inválida. Ejecutando traductor básico por defecto.")
        run_basic_translator()

Selecciona el tipo de traductor:
1: Traductor básico (palabra por palabra)
2: Traductor avanzado (con modelo de atención)
Ingresa tu opción (1 o 2): 1
Traductor básico de inglés a español (usa solo palabras conocidas)
Palabras permitidas: i, you, he, she, we, they, eat, drink, see, like, am, are, is, the, a, an, apple, water, book, house

Escribe una frase en inglés (o 'salir'): she
Entrada: she
Traducción aproximada: ella

Escribe una frase en inglés (o 'salir'): salir
